In [1]:
import pip_system_certs.wrapt_requests

# Inject system trust store into requests
pip_system_certs.wrapt_requests.inject_truststore()

In [6]:
from sentence_transformers import SentenceTransformer, util,CrossEncoder
import gradio as gr
import json
from rank_bm25 import BM25Okapi


In [7]:
corpus = [
    "How do I adopt a kitten?",
    "Best way to get a cat from a shelter",
    "Dog training tips for beginners",
    "How to learn guitar chords",
    "Guide to playing the acoustic guitar",
    "What is the capital of France?",
    "Paris is the capital city of France"
]

queries = [
    "Adopting a cat",         # embeddings > BM25
    "guitar lessons",         # embeddings > BM25
    "capital France",         # BM25 > embeddings
]


In [3]:
# BM25 setup
tokenized_corpus = [doc.lower().split() for doc in corpus]
bm25 = BM25Okapi(tokenized_corpus)


In [40]:
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import string

# Download if not already available
nltk.download("punkt")
nltk.download("stopwords")

stop_words = set(stopwords.words("english"))
punct = set(string.punctuation)

def preprocess(text):
    # Lowercase + tokenize
    tokens = word_tokenize(text.lower())
    # Remove stopwords and punctuation
    tokens = [t for t in tokens if t not in stop_words and t not in punct]
    return tokens

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\RX268UR\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\RX268UR\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\stopwords.zip.


In [41]:
# Build BM25 with preprocessed corpus
bm25 = BM25Okapi([preprocess(doc) for doc in corpus])

def search(query, top_k=5):
    # --- BM25 ---
    query_tokens = preprocess(query)
    bm25_scores = np.array(bm25.get_scores(query_tokens))
    bm25_top = np.argsort(-bm25_scores)[:top_k]

In [42]:
def search(query, top_k=3):
    print(f"\nQuery: {query}")
    
    # BM25
    bm25_scores = bm25.get_scores(query.lower().split())
    bm25_ranked = sorted(zip(corpus, bm25_scores), key=lambda x: x[1], reverse=True)[:top_k]

    
    print("\nBM25 results:")
    for doc, score in bm25_ranked:
        print(f"  ({score:.4f}) {doc}")


In [44]:
# Example queries
search("Adopting a cat")       # semantic match
search("guitar lessons")       # semantic match
search("capital France")       # keyword match


Query: Adopting a cat

BM25 results:
  (0.0000) EXHIBIT 10.6

                              DISTRIBUTOR AGREEMENT

         THIS  DISTRIBUTOR  AGREEMENT (the  "Agreement")  is made by and between Electric City Corp.,  a Delaware  corporation  ("Company")  and Electric City of Illinois LLC ("Distributor") this 7th day of September, 1999.

                                    RECITALS

         A. The  Company's  Business.  The Company is  presently  engaged in the business  of selling an energy  efficiency  device,  which is  referred to as an "Energy  Saver"  which may be improved  or  otherwise  changed  from its present composition (the "Products").  The Company may engage in the business of selling other  products  or  other  devices  other  than  the  Products,  which  will be considered  Products if Distributor  exercises its options pursuant to Section 7 hereof.

         B. Representations.  As an inducement to the Company to enter into this Agreement,  the  Distributor  has  re

In [11]:
# Embedding model
model = SentenceTransformer("all-MiniLM-L6-v2")
corpus_embeddings = model.encode(corpus, convert_to_tensor=True)

In [12]:
def search(query, top_k=3):
    print(f"\n🔎 Query: {query}")
    # Embeddings
    q_embed = model.encode(query, convert_to_tensor=True)
    cos_scores = util.cos_sim(q_embed, corpus_embeddings)[0]
    emb_ranked = sorted(zip(corpus, cos_scores), key=lambda x: x[1], reverse=True)[:top_k]

    print("\nEmbedding results:")
    for doc, score in emb_ranked:
        print(f"  ({score:.4f}) {doc}")


In [13]:
# Example queries
search("Adopting a cat")       # semantic match
search("guitar lessons")       # semantic match
search("capital France")       # keyword match


🔎 Query: Adopting a cat

Embedding results:
  (0.7266) How do I adopt a kitten?
  (0.6597) Best way to get a cat from a shelter
  (0.1772) Dog training tips for beginners

🔎 Query: guitar lessons

Embedding results:
  (0.7323) How to learn guitar chords
  (0.6780) Guide to playing the acoustic guitar
  (0.3346) Dog training tips for beginners

🔎 Query: capital France

Embedding results:
  (0.9080) What is the capital of France?
  (0.8528) Paris is the capital city of France
  (0.0511) Dog training tips for beginners


In [4]:
# Cross-encoder for reranking
cross_encoder = CrossEncoder("cross-encoder/ms-marco-MiniLM-L-6-v2")

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


In [ ]:

query = "Adopting a cat"
candidates = [
    "How do I adopt a kitten?",
    "Best way to get a cat from a shelter",
    "Dog training tips for beginners",
    "Guide to playing the acoustic guitar"
]

# Cross-encoder scores (higher = more relevant)
pairs = [(query, doc) for doc in candidates]
scores = cross_encoder.predict(pairs)

# Sort candidates by score
reranked = sorted(zip(candidates, scores), key=lambda x: x[1], reverse=True)
for doc, score in reranked:
    print(f"({score:.4f}) {doc}")


(0.0326) How do I adopt a kitten?
(-5.7128) Best way to get a cat from a shelter
(-11.3614) Guide to playing the acoustic guitar
(-11.3829) Dog training tips for beginners


In [12]:
from sentence_transformers import SentenceTransformer, CrossEncoder, util
from rank_bm25 import BM25Okapi
import numpy as np

# Corpus
corpus = [
    "How do I adopt a kitten?",
    "Best way to get a cat from a shelter",
    "Dog training tips for beginners",
    "How to learn guitar chords",
    "Guide to playing the acoustic guitar",
    "What is the capital of France?",
    "Paris is the capital city of France"
]

# BM25 setup
tokenized_corpus = [doc.lower().split() for doc in corpus]
bm25 = BM25Okapi(tokenized_corpus)

# Bi-encoder
bi_encoder = SentenceTransformer("all-MiniLM-L6-v2")
corpus_embeddings = bi_encoder.encode(corpus, convert_to_tensor=True)

# Cross-encoder for reranking
cross_encoder = CrossEncoder("cross-encoder/ms-marco-MiniLM-L-6-v2")

def hybrid_with_rerank(query, top_k=3):
    print(f"\n🔎 Query: {query}")
    
    # Step 1: BM25 candidates
    bm25_scores = np.array(bm25.get_scores(query.lower().split()))
    bm25_top = np.argsort(-bm25_scores)[:10]  # take top 10
    
    # Step 2: Bi-encoder similarity
    q_embed = bi_encoder.encode(query, convert_to_tensor=True)
    cos_scores = util.cos_sim(q_embed, corpus_embeddings)[0].cpu().numpy()
    
    # Step 3: Combine BM25 + bi-encoder (optional pre-rank)
    candidates = [(corpus[idx], cos_scores[idx]) for idx in bm25_top]
    candidates = sorted(candidates, key=lambda x: x[1], reverse=True)[:top_k*3]
    
    # Step 4: Cross-encoder rerank
    pairs = [(query, doc) for doc, _ in candidates]
    ce_scores = cross_encoder.predict(pairs)
    reranked = sorted(zip(candidates, ce_scores), key=lambda x: x[1], reverse=True)[:top_k]
    
    for (doc, _), score in reranked:
        print(f"  ({score:.4f}) {doc}")



In [13]:
# Try it
hybrid_with_rerank("Adopting a cat")
hybrid_with_rerank("guitar lessons")
hybrid_with_rerank("capital France")



🔎 Query: Adopting a cat
  (0.0326) How do I adopt a kitten?
  (-5.7128) Best way to get a cat from a shelter
  (-11.2809) Paris is the capital city of France

🔎 Query: guitar lessons
  (0.7519) How to learn guitar chords
  (-1.6675) Guide to playing the acoustic guitar
  (-11.0971) Dog training tips for beginners

🔎 Query: capital France
  (7.6821) Paris is the capital city of France
  (4.1471) What is the capital of France?
  (-11.2067) Dog training tips for beginners


In [31]:
from datasets import load_dataset

ds = load_dataset("json", data_files="data\CUADv1.json", split="train")

print(ds.column_names)
print(ds[0]["data"][0])  # CUAD stores QA in nested structure


['version', 'data']
{'title': 'LIMEENERGYCO_09_09_1999-EX-10-DISTRIBUTOR AGREEMENT', 'paragraphs': [{'qas': [{'answers': [{'text': 'DISTRIBUTOR AGREEMENT', 'answer_start': 44}], 'id': 'LIMEENERGYCO_09_09_1999-EX-10-DISTRIBUTOR AGREEMENT__Document Name', 'question': 'Highlight the parts (if any) of this contract related to "Document Name" that should be reviewed by a lawyer. Details: The name of the contract', 'is_impossible': False}, {'answers': [{'text': 'Distributor', 'answer_start': 244}, {'text': 'Electric City Corp.', 'answer_start': 148}, {'text': 'Electric City of Illinois L.L.C.', 'answer_start': 49574}, {'text': 'Company', 'answer_start': 197}, {'text': 'Electric City of Illinois LLC', 'answer_start': 212}], 'id': 'LIMEENERGYCO_09_09_1999-EX-10-DISTRIBUTOR AGREEMENT__Parties', 'question': 'Highlight the parts (if any) of this contract related to "Parties" that should be reviewed by a lawyer. Details: The two or more parties who signed the contract', 'is_impossible': False}, {'

In [32]:
from datasets import Dataset

def flatten_cuad(ds):
    rows = []
    for ex in ds:  # each ex has "version" and "data"
        for d in ex["data"]:   # go into the "data" list
            for p in d["paragraphs"]:
                context = p["context"]
                for qa in p["qas"]:
                    q = qa["question"]
                    ans = qa["answers"][0]["text"] if qa["answers"] else None
                    rows.append({
                        "context": context,
                        "question": q,
                        "answer": ans
                    })
    return Dataset.from_list(rows)

flat_ds = flatten_cuad(ds)
print(flat_ds)
print(flat_ds[0])


Dataset({
    features: ['context', 'question', 'answer'],
    num_rows: 20910
})
{'context': 'EXHIBIT 10.6\n\n                              DISTRIBUTOR AGREEMENT\n\n         THIS  DISTRIBUTOR  AGREEMENT (the  "Agreement")  is made by and between Electric City Corp.,  a Delaware  corporation  ("Company")  and Electric City of Illinois LLC ("Distributor") this 7th day of September, 1999.\n\n                                    RECITALS\n\n         A. The  Company\'s  Business.  The Company is  presently  engaged in the business  of selling an energy  efficiency  device,  which is  referred to as an "Energy  Saver"  which may be improved  or  otherwise  changed  from its present composition (the "Products").  The Company may engage in the business of selling other  products  or  other  devices  other  than  the  Products,  which  will be considered  Products if Distributor  exercises its options pursuant to Section 7 hereof.\n\n         B. Representations.  As an inducement to the Company

In [37]:
# Save locally (fast binary format)
flat_ds.save_to_disk("cuad_flat")

# Later reload instantly
from datasets import load_from_disk
flat_ds = load_from_disk("cuad_flat")
print(flat_ds)


Saving the dataset (3/3 shards): 100%|██████████| 20910/20910 [00:02<00:00, 9169.53 examples/s]


Dataset({
    features: ['context', 'question', 'answer'],
    num_rows: 20910
})


In [38]:
from datasets import load_from_disk

# Load your saved CUAD flat dataset
flat_ds = load_from_disk("cuad_flat")

# Take first 100 rows for quick experiments
small_ds = flat_ds.select(range(min(100, len(flat_ds))))

# Prepare corpus (contexts) and queries (questions)
corpus = small_ds["context"]
queries = small_ds["question"]

print(f"Corpus size: {len(corpus)}")
print("Sample query:", queries[0])
print("Sample context:", corpus[0][:300], "...")


Corpus size: 100
Sample query: Highlight the parts (if any) of this contract related to "Document Name" that should be reviewed by a lawyer. Details: The name of the contract
Sample context: EXHIBIT 10.6

                              DISTRIBUTOR AGREEMENT

         THIS  DISTRIBUTOR  AGREEMENT (the  "Agreement")  is made by and between Electric City Corp.,  a Delaware  corporation  ("Company")  and Electric City of Illinois LLC ("Distributor") this 7th day of September, 1999.

         ...


In [33]:
# take first 100 rows
small_ds = flat_ds.select(range(100))
corpus = list(small_ds["context"])
queries = list(small_ds["question"])

In [35]:
print(corpus[0])

EXHIBIT 10.6

                              DISTRIBUTOR AGREEMENT

         THIS  DISTRIBUTOR  AGREEMENT (the  "Agreement")  is made by and between Electric City Corp.,  a Delaware  corporation  ("Company")  and Electric City of Illinois LLC ("Distributor") this 7th day of September, 1999.

                                    RECITALS

         A. The  Company's  Business.  The Company is  presently  engaged in the business  of selling an energy  efficiency  device,  which is  referred to as an "Energy  Saver"  which may be improved  or  otherwise  changed  from its present composition (the "Products").  The Company may engage in the business of selling other  products  or  other  devices  other  than  the  Products,  which  will be considered  Products if Distributor  exercises its options pursuant to Section 7 hereof.

         B. Representations.  As an inducement to the Company to enter into this Agreement,  the  Distributor  has  represented  that  it has or  will  have  the facil

In [36]:
print(queries[0])

Highlight the parts (if any) of this contract related to "Document Name" that should be reviewed by a lawyer. Details: The name of the contract


In [39]:
import pandas as pd

data = [
    {"id": 1, "clause_type": "Termination", "context": "Either party may terminate this Agreement with 30 days’ prior written notice."},
    {"id": 2, "clause_type": "Termination", "context": "The contract shall terminate automatically in case of material breach by either party."},
    {"id": 3, "clause_type": "Renewal", "context": "This Agreement shall automatically renew for successive one-year terms unless either party objects."},
    {"id": 4, "clause_type": "Renewal", "context": "Renewal requires mutual written consent of both parties prior to expiration."},
    {"id": 5, "clause_type": "Confidentiality", "context": "The parties agree to maintain the confidentiality of all proprietary information exchanged."},
    {"id": 6, "clause_type": "Confidentiality", "context": "Confidential information shall not be disclosed except as required by law."},
    {"id": 7, "clause_type": "Payment", "context": "Payments are due within 30 days of invoice receipt."},
    {"id": 8, "clause_type": "Payment", "context": "Late payments will incur a penalty of 2% per month."},
    {"id": 9, "clause_type": "Liability", "context": "Neither party shall be liable for indirect or consequential damages."},
    {"id": 10, "clause_type": "Liability", "context": "The total liability of either party shall not exceed the fees paid under this Agreement."},
    {"id": 11, "clause_type": "Governing Law", "context": "This Agreement shall be governed by the laws of the State of Delaware."},
    {"id": 12, "clause_type": "Governing Law", "context": "Any disputes shall be resolved under the jurisdiction of New York courts."},
    {"id": 13, "clause_type": "Force Majeure", "context": "Neither party shall be liable for delays caused by natural disasters or other force majeure events."},
    {"id": 14, "clause_type": "Force Majeure", "context": "Performance shall be excused in case of war, strikes, or government restrictions."},
    {"id": 15, "clause_type": "Dispute Resolution", "context": "Any disputes shall be resolved through binding arbitration in accordance with AAA rules."},
    {"id": 16, "clause_type": "Dispute Resolution", "context": "Parties agree to first attempt mediation before pursuing arbitration."},
    {"id": 17, "clause_type": "Assignment", "context": "Neither party may assign this Agreement without prior written consent of the other party."},
    {"id": 18, "clause_type": "Assignment", "context": "Assignment is permitted to affiliates with prior notice."},
    {"id": 19, "clause_type": "Warranties", "context": "The supplier warrants that the products shall conform to specifications for 12 months."},
    {"id": 20, "clause_type": "Warranties", "context": "No additional warranties are provided unless expressly stated in writing."},
]

df = pd.DataFrame(data)
df.to_csv("contract_clauses_demo.csv", index=False)
print("Saved contract_clauses_demo.csv with 20 clauses")


Saved contract_clauses_demo.csv with 20 clauses


In [49]:
import string
import numpy as np
import pandas as pd
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from rank_bm25 import BM25Okapi
from sentence_transformers import SentenceTransformer, util, CrossEncoder
from datasets import Dataset

# --- Preprocessing ---
stop_words = set(stopwords.words("english"))
punct = set(string.punctuation)

def preprocess(text):
    tokens = [t.lower() for t in word_tokenize(text)
              if t.lower() not in stop_words and t not in punct]
    return tokens

# --- Load contract dataset ---
df = pd.read_csv("contract_clauses_demo.csv")
contract_ds = Dataset.from_pandas(df)

small_ds = contract_ds.select(range(min(20, len(contract_ds))))
corpus = small_ds["context"]

# --- Prepare BM25 ---
tokenized_corpus = [preprocess(doc) for doc in corpus]
bm25 = BM25Okapi(tokenized_corpus)

# --- Bi-encoder ---
bi_encoder = SentenceTransformer("all-MiniLM-L6-v2")
corpus_embeds = bi_encoder.encode(corpus, convert_to_tensor=True)

# --- Cross-encoder ---
cross_encoder = CrossEncoder("cross-encoder/ms-marco-MiniLM-L-6-v2")

# --- Search function ---
def search(query, top_k=5):
    # Preprocess query for BM25
    q_tokens = preprocess(query)
    bm25_scores = np.array(bm25.get_scores(q_tokens))
    bm25_top = np.argsort(-bm25_scores)[:top_k]

    # Bi-encoder
    q_embed = bi_encoder.encode(query, convert_to_tensor=True)
    cos_scores = util.cos_sim(q_embed, corpus_embeds)[0].cpu().numpy()
    bi_top = np.argsort(-cos_scores)[:top_k]

    # Merge candidates
    candidates = set(bm25_top) | set(bi_top)
    pairs = [(query, corpus[idx]) for idx in candidates]

    # Cross-encoder rerank
    ce_scores = cross_encoder.predict(pairs)
    reranked = sorted(zip([c[1] for c in pairs], ce_scores),
                      key=lambda x: x[1], reverse=True)

    return {
        "BM25": [(corpus[i], bm25_scores[i]) for i in bm25_top],
        "Bi-Encoder": [(corpus[i], cos_scores[i]) for i in bi_top],
        "Cross-Encoder": reranked[:top_k]
    }

# --- Test with a query ---
query = "act of god"
print("🔎 Query:", query)
results = search(query, top_k=3)

print("\nBM25 results:")
for doc, score in results["BM25"]:
    print(f"  ({score:.4f}) {doc[:200]}...")

print("\nBi-Encoder results:")
for doc, score in results["Bi-Encoder"]:
    print(f"  ({score:.4f}) {doc[:200]}...")

print("\nCross-Encoder reranked results:")
for doc, score in results["Cross-Encoder"]:
    print(f"  ({score:.4f}) {doc[:500]}...")


🔎 Query: act of god

BM25 results:
  (0.0000) Either party may terminate this Agreement with 30 days’ prior written notice....
  (0.0000) The contract shall terminate automatically in case of material breach by either party....
  (0.0000) This Agreement shall automatically renew for successive one-year terms unless either party objects....

Bi-Encoder results:
  (0.1953) Performance shall be excused in case of war, strikes, or government restrictions....
  (0.1380) Neither party shall be liable for delays caused by natural disasters or other force majeure events....
  (0.1350) Any disputes shall be resolved under the jurisdiction of New York courts....

Cross-Encoder reranked results:
  (-11.2440) The contract shall terminate automatically in case of material breach by either party....
  (-11.2526) Neither party shall be liable for delays caused by natural disasters or other force majeure events....
  (-11.2538) This Agreement shall automatically renew for successive one-year terms unl